In [1]:
# ==============================================================================
# Slack Data Processing - Convert Export to Training Dataset
# ==============================================================================

import sys
import os
from pathlib import Path

# Add current directory to path to import our processor
sys.path.append('.')

# Import our custom Slack data processor
try:
    from slack_data_processor import SlackDataProcessor
    print("Slack data processor imported successfully!")
except ImportError as e:
    print(f"Error importing slack_data_processor: {e}")
    print("Make sure slack_data_processor.py is in the same directory as this notebook")

def process_slack_data_interactive():
    """
    Interactive function to process Slack data for training
    """
    print("=== Slack Data Processing for LoRA Training ===\n")
    
    # Check if sample data exists
    sample_path = Path("./sampledata/slack")
    if sample_path.exists():
        print(f"Found sample data directory: {sample_path}")
        
        # List available export directories
        export_dirs = [d for d in sample_path.iterdir() if d.is_dir()]
        
        if export_dirs:
            print("\nAvailable Slack export directories:")
            for i, export_dir in enumerate(export_dirs, 1):
                print(f"{i}. {export_dir.name}")
            
            # Let user select directory
            try:
                choice = int(input(f"\nSelect export directory (1-{len(export_dirs)}): ")) - 1
                if 0 <= choice < len(export_dirs):
                    selected_dir = export_dirs[choice]
                    print(f"Selected: {selected_dir}")
                    
                    # Process the data
                    processor = SlackDataProcessor(str(selected_dir))
                    result = processor.process("./processed_datasets")
                    
                    if result:
                        print("\n🎉 Success! Your dataset is ready for training!")
                        print(f"Load this CSV file in the training cell: {result['csv']}")
                        return result
                    else:
                        print("❌ Processing failed.")
                        return None
                else:
                    print("Invalid selection.")
                    return None
            except ValueError:
                print("Please enter a valid number.")
                return None
        else:
            print("No export directories found in sample data.")
            return None
    else:
        print(f"Sample data directory not found: {sample_path}")
        print("Please place your Slack export in the sampledata/slack/ directory")
        return None

def process_custom_slack_path(slack_path: str):
    """
    Process Slack data from a custom path
    """
    if not os.path.exists(slack_path):
        print(f"Path does not exist: {slack_path}")
        return None
    
    processor = SlackDataProcessor(slack_path)
    return processor.process("./processed_datasets")

# Example usage:
print("Available functions:")
print("1. process_slack_data_interactive() - Interactive processing of data in sampledata/slack/")
print("2. process_custom_slack_path('path/to/slack/export') - Process custom path")
print("\nRun one of these functions to start processing your Slack data!")

Slack data processor imported successfully!
Available functions:
1. process_slack_data_interactive() - Interactive processing of data in sampledata/slack/
2. process_custom_slack_path('path/to/slack/export') - Process custom path

Run one of these functions to start processing your Slack data!


# LoRA Persona Model Training - Local RTX A4000 Version

This notebook is optimized for running locally on RTX A4000 GPU. It builds upon the original Colab notebook but with modifications for local execution:

- Optimized memory usage for RTX A4000 (16GB VRAM)
- Local file paths and data handling
- Better error handling and logging
- CUDA optimizations for local GPU

In [ ]:
# ==============================================================================
# Phase 0: Environment Setup for Local RTX A4000
# ==============================================================================

import os
import sys
import torch
import warnings
warnings.filterwarnings('ignore')

print("Phase 0: Setting up environment for local RTX A4000...")
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("WARNING: CUDA not available. This notebook requires GPU acceleration.")

# Install required packages if not already installed
try:
    import transformers
    import datasets
    import peft
    import accelerate
    import bitsandbytes
    import trl
    print("All required packages are already installed.")
except ImportError as e:
    print(f"Missing package: {e}")
    print("Please install missing packages using:")
    print("pip install transformers datasets peft accelerate bitsandbytes trl sentencepiece")

print("Environment setup completed.")

In [ ]:
# ==============================================================================
# Phase 1: Base Model and Tokenizer Setup (Optimized for RTX A4000)
# ==============================================================================

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import gc

print("\nPhase 1: Preparing base model and tokenizer...")

# Clear any existing GPU memory
torch.cuda.empty_cache()
gc.collect()

# Model selection - you can change this to other models
# Options for better local performance:
# - "microsoft/DialoGPT-medium" (English, smaller)
# - "rinna/japanese-gpt-neox-3.6b" (Japanese, smaller)
# - "stabilityai/japanese-stablelm-instruct-gamma-7b" (Japanese, larger but good quality)
model_id = "stabilityai/japanese-stablelm-instruct-gamma-7b"

# Optimized quantization config for RTX A4000
# RTX A4000 has 16GB VRAM, so we can be less aggressive with quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for better performance on RTX A4000
)

print(f"Loading model: {model_id}")
print("This may take several minutes...")

# Load model with optimized settings for local GPU
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,  # Optimize for RTX A4000
    low_cpu_mem_usage=True,      # Optimize CPU memory usage
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Base model and tokenizer setup completed.")
print(f"Model: {model_id}")
print(f"GPU Memory Used: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"GPU Memory Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

In [ ]:
# ==============================================================================
# Phase 2: Training Data Preparation (Local File Support)
# ==============================================================================

from datasets import Dataset
import pandas as pd
import json
from pathlib import Path

print("\nPhase 2: Preparing training data...")

# ==============================================================================
# Data Loading Options - Choose one of the following methods:
# ==============================================================================

# Option 1: Load processed dataset from slack_data_processor.py
# RECOMMENDED: Use this after running the Slack Data Processing cell above
processed_datasets_dir = Path("./processed_datasets")
if processed_datasets_dir.exists():
    # Find the most recent CSV file
    csv_files = list(processed_datasets_dir.glob("slack_persona_*.csv"))
    if csv_files:
        # Sort by modification time, get most recent
        latest_csv = max(csv_files, key=lambda x: x.stat().st_mtime)
        print(f"Loading processed dataset: {latest_csv}")
        
        df_loaded = pd.read_csv(latest_csv)
        my_slack_data = df_loaded.to_dict('records')
        print(f"✅ Loaded {len(my_slack_data)} training examples from processed dataset")
        
        # Show sample of loaded data
        print("\n--- Sample from loaded dataset ---")
        print(f"Input: {my_slack_data[0]['input'][:100]}...")
        print(f"Output: {my_slack_data[0]['output'][:100]}...")
        print("----------------------------------")
    else:
        print("No processed CSV files found. Using sample data instead.")
        print("💡 Run the Slack Data Processing cell first to create your dataset!")
        csv_files = None
else:
    print("Processed datasets directory not found. Using sample data.")
    csv_files = None

# Option 2: Load from custom JSON/CSV file
# Uncomment and modify the path below if you have a custom file
# custom_file = Path("./path/to/your/custom_data.csv")  # or .json
# if custom_file.exists():
#     if custom_file.suffix == '.csv':
#         df_loaded = pd.read_csv(custom_file)
#         my_slack_data = df_loaded.to_dict('records')
#     else:  # JSON
#         with open(custom_file, 'r', encoding='utf-8') as f:
#             my_slack_data = json.load(f)
#     print(f"Loaded {len(my_slack_data)} records from {custom_file}")
#     csv_files = [custom_file]  # Mark as loaded

# Option 3: Sample data (fallback)
# This will be used if no processed data is found
if not csv_files:
    print("⚠️  Using sample data. For better results, process your actual Slack data first!")
    my_slack_data = [
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーA: 「来週の定例会議ですが、アジェンダ案を共有します。何か追加項目はありますか？」",
        "output": "ありがとうございます！拝見しました。可能であれば、先日話していた新機能の進捗についても5分ほど時間をいただけると嬉しいです。"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーB: 「この前の件、調査してみたんですが、原因はサーバー側の設定ミスだったみたいです。すみません…。」",
        "output": "調査ありがとうございます！原因が特定できてよかったです。誰にでもミスはありますし、気にしないでください。すぐに対応してくれて助かりました！"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーC: 「新しいAIの論文読んだ？すごく面白かったよ。」",
        "output": "読みました！特にエージェントが自律的に協調する部分、示唆に富んでますよね。あれを私たちのプロダクトに応用できないか、少し考えてみたくなります。"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーD: 「今度のプロジェクト、スケジュールがタイトですね。どう思います？」",
        "output": "確かにタイトですね。まずは最重要な機能を特定して、段階的にリリースする方針はどうでしょうか？品質を保ちつつ、現実的なスケジュールで進められると思います。"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "input": "ユーザーE: 「お疲れ様です。今日のデモ、とても良かったです！」",
        "output": "お疲れ様でした！ありがとうございます。チーム全員で準備した甲斐がありました。特にどの部分が印象的でしたか？今後の参考にさせていただきたいです。"
    }
]

# Create DataFrame and display sample
df = pd.DataFrame(my_slack_data)
print("\n--- Training Data Sample ---")
print(df.head())
print("----------------------------")
print(f"Total data records: {len(df)}")

if len(df) < 50:
    print("\nWARNING: You have very few training examples. For better results, consider adding more data.")
    print("Recommended: 100+ examples for decent results, 300+ for good results.")

# Create prompt formatting function
def create_prompt(data_point):
    prompt = f"""以下は、タスクを説明する指示と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{data_point["instruction"]}

### 入力:
{data_point["input"]}

### 応答:
{data_point["output"]}"""
    return {"text": prompt}

# Create dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(create_prompt)

print("\nTraining data preparation completed.")
print(f"Dataset size: {len(dataset)} examples")

# Save processed data locally for future use
output_dir = Path("./training_data")
output_dir.mkdir(exist_ok=True)
dataset.save_to_disk(str(output_dir / "processed_dataset"))
print(f"Processed dataset saved to: {output_dir / 'processed_dataset'}")

In [ ]:
# ==============================================================================
# Phase 3: LoRA Fine-tuning (Optimized for RTX A4000)
# ==============================================================================

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer
import os
from datetime import datetime

print("\nPhase 3: Starting LoRA fine-tuning...")

# Clear GPU memory before training
torch.cuda.empty_cache()
gc.collect()

# LoRA configuration optimized for RTX A4000
lora_config = LoraConfig(
    r=16,  # Rank - balance between capability and memory usage
    lora_alpha=32,  # Scaling factor
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"  # Additional modules for better coverage
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

# Prepare model for training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

# Create output directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"./lora-output-{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Training arguments optimized for RTX A4000
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,  # Start with 3 epochs, adjust based on results
    per_device_train_batch_size=4,  # RTX A4000 can handle larger batches
    gradient_accumulation_steps=2,   # Effective batch size = 4 * 2 = 8
    optim="paged_adamw_8bit",        # Memory-efficient optimizer
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,  # RTX A4000 supports bfloat16 better
    bf16=True,   # Use bfloat16 for better numerical stability
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    logging_steps=1,
    logging_dir=f"{output_dir}/logs",
    save_strategy="epoch",
    save_steps=50,
    evaluation_strategy="no",  # Disable evaluation to save memory
    save_total_limit=3,
    load_best_model_at_end=False,
    ddp_find_unused_parameters=False,
    dataloader_num_workers=0,  # Reduce CPU usage
    report_to="none",  # Disable wandb/tensorboard for local training
    run_name=f"lora-persona-{timestamp}",
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    max_seq_length=1024,  # Adjust based on your data
    tokenizer=tokenizer,
    args=training_args,
    packing=False,  # Disable packing for better control
)

# Start training
print("\n=== Training Started ===")
print(f"Output directory: {output_dir}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total training steps: {len(dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")
print("\nStarting training...")

# Monitor GPU memory before training
print(f"GPU Memory before training: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

trainer.train()

print("\n=== Training Completed ===")

# Save the trained model
model_save_path = f"./lora-persona-model-{timestamp}"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Trained LoRA adapter saved to: {model_save_path}")
print(f"Training logs saved to: {output_dir}/logs")

# Save training info
training_info = {
    "model_id": model_id,
    "dataset_size": len(dataset),
    "num_epochs": training_args.num_train_epochs,
    "learning_rate": training_args.learning_rate,
    "lora_rank": lora_config.r,
    "lora_alpha": lora_config.lora_alpha,
    "timestamp": timestamp,
    "save_path": model_save_path
}

with open(f"{model_save_path}/training_info.json", 'w', encoding='utf-8') as f:
    json.dump(training_info, f, ensure_ascii=False, indent=2)

print("Training information saved.")

In [ ]:
# ==============================================================================
# Phase 4: Inference Testing (Local Optimized)
# ==============================================================================

from peft import PeftModel
import time

print("\nPhase 4: Testing the trained persona model...")

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Load base model for inference
print("Loading base model for inference...")
inference_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

# Load the trained LoRA adapter
print(f"Loading trained LoRA adapter from: {model_save_path}")
inference_model = PeftModel.from_pretrained(inference_model, model_save_path)
inference_model.eval()

print("Model loaded successfully for inference.")

# Response generation function with improved parameters
def generate_response(instruction, context, max_new_tokens=256, temperature=0.7):
    prompt = f"""以下は、タスクを説明する指示と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{instruction}

### 入力:
{context}

### 応答:
"""
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=768)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    # Generate response
    start_time = time.time()
    with torch.no_grad():
        outputs = inference_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.05,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generation_time = time.time() - start_time
    
    # Decode and extract response
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the response part
    if "### 応答:" in generated_text:
        response = generated_text.split("### 応答:")[1].strip()
    else:
        response = "[応答の抽出に失敗しました]"
    
    return response, generation_time

# Test cases
test_cases = [
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "context": "ユーザーF: 「急で申し訳ないんだけど、この資料のレビューお願いできないかな？ 明日の朝までだと嬉しいんだけど…。」"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "context": "ユーザーG: 「新しい技術について調べているんですが、どう思いますか？"
    },
    {
        "instruction": "以下の対話の文脈に続いて、あなたらしく返信を生成してください。",
        "context": "ユーザーH: 「今度のプレゼン、緊張するなあ。何かアドバイスありますか？"
    }
]

print("\n=== Inference Testing ===")
total_time = 0

for i, test_case in enumerate(test_cases, 1):
    print(f"\n--- Test Case {i} ---")
    print(f"Context: {test_case['context']}")
    
    response, gen_time = generate_response(
        test_case["instruction"], 
        test_case["context"]
    )
    
    print(f"\nGenerated Response: {response}")
    print(f"Generation time: {gen_time:.2f} seconds")
    total_time += gen_time

print(f"\n=== Testing Completed ===")
print(f"Average generation time: {total_time / len(test_cases):.2f} seconds")
print(f"GPU Memory Used: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

# Interactive testing function
def interactive_test():
    print("\n=== Interactive Testing Mode ===")
    print("Enter 'quit' to exit")
    
    while True:
        user_input = input("\nEnter context for testing: ")
        if user_input.lower() == 'quit':
            break
            
        instruction = "以下の対話の文脈に続いて、あなたらしく返信を生成してください。"
        response, gen_time = generate_response(instruction, user_input)
        print(f"\nResponse: {response}")
        print(f"Time: {gen_time:.2f}s")

print("\nYou can now test your model interactively by calling: interactive_test()")
print("All training and inference setup completed successfully!")

In [ ]:
# ==============================================================================
# Bonus: Model Management and Utilities
# ==============================================================================

import os
import shutil
from pathlib import Path

def list_saved_models():
    """List all saved LoRA models in the current directory"""
    models = []
    for item in Path(".").iterdir():
        if item.is_dir() and "lora-persona-model" in item.name:
            models.append(item)
    
    print("Saved LoRA models:")
    for i, model in enumerate(models, 1):
        print(f"{i}. {model.name}")
        # Try to read training info if available
        info_file = model / "training_info.json"
        if info_file.exists():
            with open(info_file, 'r') as f:
                info = json.load(f)
                print(f"   - Dataset size: {info.get('dataset_size', 'Unknown')}")
                print(f"   - Epochs: {info.get('num_epochs', 'Unknown')}")
                print(f"   - Learning rate: {info.get('learning_rate', 'Unknown')}")
    
    return models

def cleanup_old_models(keep_latest=2):
    """Clean up old model directories, keeping only the latest N models"""
    models = list_saved_models()
    if len(models) <= keep_latest:
        print(f"Only {len(models)} models found, no cleanup needed.")
        return
    
    # Sort by modification time
    models.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    
    models_to_delete = models[keep_latest:]
    print(f"Deleting {len(models_to_delete)} old models...")
    
    for model in models_to_delete:
        print(f"Deleting: {model.name}")
        shutil.rmtree(model)
    
    print("Cleanup completed.")

def show_gpu_memory():
    """Display current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        
        print(f"GPU Memory Status:")
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved:  {reserved:.2f} GB")
        print(f"  Total:     {total:.2f} GB")
        print(f"  Free:      {total - reserved:.2f} GB")
    else:
        print("CUDA not available")

def clear_gpu_cache():
    """Clear GPU cache to free up memory"""
    torch.cuda.empty_cache()
    gc.collect()
    print("GPU cache cleared.")
    show_gpu_memory()

# Show available utilities
print("Available utility functions:")
print("- list_saved_models(): List all saved LoRA models")
print("- cleanup_old_models(keep_latest=2): Clean up old models")
print("- show_gpu_memory(): Show current GPU memory usage")
print("- clear_gpu_cache(): Clear GPU cache")
print("- interactive_test(): Start interactive testing mode")

# Show current status
print("\nCurrent status:")
show_gpu_memory()
list_saved_models()